# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

ModuleNotFoundError: No module named 'gmaps'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
path = '../output_data/cities.csv'

cities = pd.read_csv(path)

cities.head()


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
gmaps.configure(api_key=g_key)

locations = cities[['Lat', 'Lng']]
humidity = cities['Humidity'].astype(int)

# Layout
figure_layout = {
     'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)
# Add layer
fig.add_layer(heat_layer)

fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
my_vac = cities.loc[((cities['Max Temp'] >70) & (cities ['Max Temp'] <85)) & 
                        (cities['Wind Speed'] <=5) & (cities['Cloudiness']==0)].dropna()
my_vac

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:

ideal = my_vac.copy()
ideal['Hotel Name'] = ""
ideal

In [ ]:

hotels = []

# target_coordinates = list(zip(ideal.Lat, ideal.Lng))
target_radius = 5000
target_type = 'lodging'

base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for i,row in ideal.iterrows():
    params = {
    'location': (f"{row['Lat']},{row['Lng']}"),
    'radius': target_radius,
    'types': target_type,
    'key': g_key
}
    response = requests.get(base_url, params=params).json()
#     print(response['results'][0]['name'])
    
    try:
        ideal.loc[i,'Hotel Name'] = response['results'][0]['name']
    except (KeyError, IndexError):
        print('not found')
        
ideal

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add layer
fig.add_layer(marker_layer)

# Display figure
fig
